# Librarii

In [5]:
import cv2 as cv
import numpy as np
import os

# Citirea datelor

In [6]:
director = 'testare'
files = os.listdir(director)
files.sort()

# Functii necesare

In [7]:
def extrage_careu(image):
    image_copy = image.copy()
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 3) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.47, image_g_blur, -0.9, 0)
    #show_image('image_sharpened',image_sharpened)
    _, thresh = cv.threshold(image_sharpened, 115, 255, cv.THRESH_BINARY)

    kernel = np.ones((8, 8), np.uint8)
    thresh = cv.dilate(thresh, kernel)
    #show_image('image_thresholded',thresh)

    edges =  cv.Canny(thresh ,200,400)
    edges = cv.dilate(edges, kernel)
    #show_image('edges',edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    #cv.drawContours(image, contours, -1, (0,255,255), 6)
    #show_image('contours',image)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2700
    height = 2700
    
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    #show_image("detected corners",image_copy)

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(image, M, (width, height))
    #result = cv.cvtColor(result,cv.COLOR_GRAY2BGR)
    
    return result

In [8]:
def determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical):
    matrix = np.empty((15,15), dtype='str')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 24
            y_max = lines_vertical[j + 1][1][0] - 24
            x_min = lines_horizontal[i][0][1] + 24
            x_max = lines_horizontal[i + 1][1][1] - 24
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            #show_image('image', patch)
            Medie_patch=np.mean(patch)
            if Medie_patch > 5:
                matrix[i][j]='x'
            else:
                matrix[i][j]='o'
    return matrix

In [9]:
def creaza_template ():
    img = cv.imread('imagini_auxiliare/litere_1.jpg')
    result = extrage_careu(img)
    index = 0
    templates = []
    for i in range(5, 10):
        for j in range(5, 10):
            y_min = lines_vertical[j][0][0] + 20
            y_max = lines_vertical[j + 1][1][0] - 20
            x_min = lines_horizontal[i][0][1] + 20
            x_max = lines_horizontal[i + 1][1][1] - 20
            patch = result[x_min:x_max, y_min:y_max].copy()
            templates.append(patch)
            #show_image('template', patch)
            index += 1
            
    return templates


In [10]:
def clasifica_litera(patch, templates):
        maxi=-np.inf
        poz=-1
        
        image_m_blur = cv.medianBlur(patch,3)
        image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 3) 
        patch = cv.addWeighted(image_m_blur, 2, image_g_blur, -1, 0)
        height = patch.shape[0]
        width = patch.shape[1]
        #patch = patch[25:width-25, 25:height-25]
        #_, patch = cv.threshold(patch, 90, 255, cv.THRESH_BINARY)
        
        #show_image('patch', patch)
        
        for j in range(0,23):
            img_template = templates[j]
            #img_template = cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
            image_m_blur = cv.medianBlur(img_template,3)
            image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 3) 
            img_template = cv.addWeighted(image_m_blur, 2, image_g_blur, -1, 0)
            #_, img_template = cv.threshold(img_template, 90, 255, cv.THRESH_BINARY)
            
            height = img_template.shape[0]
            width = img_template.shape[1]
            img_template = img_template[30:width-30, 30:height-30]
            
            #show_image('template', img_template)
            
            corr = cv.matchTemplate(patch, img_template,  cv.TM_CCOEFF_NORMED)
            corr=np.max(corr)
            if corr>maxi:
                maxi=corr
                poz=j
        return poz

In [11]:
def determina_configuratie_careu_olitere(img,thresh,lines_horizontal,lines_vertical, templates):
    matrix = np.empty((15,15), dtype='str')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 24
            y_max = lines_vertical[j + 1][1][0] - 24
            x_min = lines_horizontal[i][0][1] + 24
            x_max = lines_horizontal[i + 1][1][1] - 24
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            patch_orig=img[x_min-24:x_max+24, y_min-24:y_max+24].copy()
            #patch_orig= cv.cvtColor(patch_orig,cv.COLOR_BGR2GRAY)
            Medie_patch=np.mean(patch)
            if Medie_patch>5:
                lit = clasifica_litera(patch_orig, templates)
                matrix[i][j] = litere[lit]
                
            else:
                matrix[i][j]=' '
    return matrix

# Hardcode tabla joc, litere si punctaje

In [15]:
lines_horizontal=[]
for i in range(0,2701,180):
    l=[]
    l.append((0,i))
    l.append((2699,i))
    lines_horizontal.append(l)

In [16]:
lines_vertical=[]
for i in range(0,2701,180):
    l=[]
    l.append((i,0))
    l.append((i,2699))
    lines_vertical.append(l)

In [17]:
tabla = [
    [23, 0, 0, 12, 0, 0, 0, 23, 0, 0, 0, 12, 0, 0, 23],
    [0, 22, 0, 0, 0, 13, 0, 0, 0, 13, 0, 0, 0, 22, 0],
    [0, 0, 22, 0, 0, 0, 12, 0, 12, 0, 0, 0, 22, 0, 0],
    [12, 0, 0, 22, 0, 0, 0, 12, 0, 0, 0, 22, 0, 0, 12],
    [0, 0, 0, 0, 22, 0, 0, 0, 0, 0, 22, 0, 0, 0, 0],
    [0, 13, 0, 0, 0, 13, 0, 0, 0, 13, 0, 0, 0, 13, 0],
    [0, 0, 12, 0, 0, 0, 12, 0, 12, 0, 0, 0, 12, 0, 0],
    [23, 0, 0, 12, 0, 0, 0, 22, 0, 0, 0, 12, 0, 0, 23],
    [0, 0, 12, 0, 0, 0, 12, 0, 12, 0, 0, 0, 12, 0, 0],
    [0, 13, 0, 0, 0, 13, 0, 0, 0, 13, 0, 0, 0, 13, 0],
    [0, 0, 0, 0, 22, 0, 0, 0, 0, 0, 22, 0, 0, 0, 0],
    [12, 0, 0, 22, 0, 0, 0, 12, 0, 0, 0, 22, 0, 0, 12],
    [0, 0, 22, 0, 0, 0, 12, 0, 12, 0, 0, 0, 22, 0, 0],
    [0, 22, 0, 0, 0, 13, 0, 0, 0, 13, 0, 0, 0, 22, 0],
    [23, 0, 0, 12, 0, 0, 0, 23, 0, 0, 0, 12, 0, 0, 23],
]

litere = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', 'Z', '?']
puncte = {
    'A': 1,
    'B': 9,
    'C': 1,
    'D': 2,
    'E': 1,
    'F': 8,
    'G': 9,
    'H': 10,
    'I': 1,
    'J': 10,
    'L': 1,
    'M': 4,
    'N': 1,
    'O': 1,
    'P': 2,
    'R': 1,
    'S': 1,
    'T': 1,
    'U': 1,
    'V': 8,
    'X': 10,
    'Z': 10,
    '?': 0
}

templates = creaza_template()


# Task 1

In [14]:
def afisare_task_1(lista, file):
    path_solutie = 'fisiere_solutie'
    if not os.path.exists(path_solutie):
        os.makedirs(path_solutie)
        
    solutie = ""
    
    for elem in lista:
        solutie += str(elem[0] + 1) + chr(65 + elem[1]) + '\n'
    
    with open(os.path.join(path_solutie, file[:-3] + 'txt'), 'w') as fout:
        fout.write(solutie)

In [17]:
index = 0

for file in files:
    if file[-3:]=='jpg':
        img = cv.imread(director+'/'+file)
        result=extrage_careu(img)
        _, thresh = cv.threshold(result, 190, 255, cv.THRESH_BINARY)
        matrice=determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical)
        #vizualizare_configuratie(result,matrice,lines_horizontal,lines_vertical)
        
        if index == 0:
            matrice_ant = np.full((15,15), 'o')
        
        lit_noi = []
        for i in range(15):
            for j in range(15): 
                if matrice[i][j] != matrice_ant[i][j]:
                    lit_noi.append([i, j])
                if len(lit_noi) > 6:
                    i = 15
                    break
                    
        afisare_task_1(lit_noi, file)
        
        index += 1
        index %= 20
        
        matrice_ant = matrice
        
        #show_image('img',result)

KeyboardInterrupt: 

# Task 2

In [18]:
def afisare_task_2(lista, file, matrice):
    path_solutie = 'fisiere_solutie'
    if not os.path.exists(path_solutie):
        os.makedirs(path_solutie)
        
    solutie = ""
    
    for elem in lista:
        solutie += str(elem[0] + 1) + chr(65 + elem[1]) + ' ' + matrice[elem[0]][elem[1]] + '\n'
    
    with open(os.path.join(path_solutie, file[:-3] + 'txt'), 'w') as fout:
        fout.write(solutie)

In [19]:
index = 0

for file in files:
    if file[-3:]=='jpg':
        img = cv.imread(director+'/'+file)
        result=extrage_careu(img)
        _, thresh = cv.threshold(result, 190, 255, cv.THRESH_BINARY)
        matrice=determina_configuratie_careu_olitere(result,thresh,lines_horizontal,lines_vertical, templates)
        
        if index == 0:
            matrice_ant = np.full((15,15), ' ')
        
        lit_noi = []
        for i in range(15):
            for j in range(15):
                if matrice[i][j] != matrice_ant[i][j]:
                    lit_noi.append([i, j])
                if len(lit_noi) > 6:
                    i = 15
                    break
                    
        afisare_task_2(lit_noi, file, matrice)
        
        index += 1
        index %= 20
        
        matrice_ant = matrice

KeyboardInterrupt: 

# Task 3

In [18]:
def afisare_task_3(scor, matrice, lista, file):
    path_solutie = 'fisiere_solutie'
    if not os.path.exists(path_solutie):
        os.makedirs(path_solutie)
        
    solutie = ""
    
    for elem in lista:
        solutie += str(elem[0] + 1) + chr(65 + elem[1]) + ' ' + matrice[elem[0]][elem[1]] + '\n'
    solutie += str(scor)
    
    with open(os.path.join(path_solutie, file[:-3] + 'txt'), 'w') as fout:
        fout.write(solutie)


In [19]:
index = 0

for file in files:
    if file[-3:]=='jpg':
        img = cv.imread(director+'/'+file)
        result = extrage_careu(img)
        _, thresh = cv.threshold(result, 190, 255, cv.THRESH_BINARY)
        matrice = determina_configuratie_careu_olitere(result,thresh,lines_horizontal,lines_vertical, templates)
        if index == 0:
            matrice_ant = np.full((15,15), ' ')
            
        scor = 0
        
        lit_noi = []
        for i in range(15):
            for j in range(15):
                if matrice[i][j] != matrice_ant[i][j]:
                    lit_noi.append([i, j])
                if len(lit_noi) > 6:
                    scor += 50
                    i = 15
                    break
        #print(len(lit_noi))
        if index == 0:
            for lit in lit_noi:
                scor += puncte[matrice[lit[0]][lit[1]]]
                if tabla[lit[0]][lit[1]] == 12:
                    scor += puncte[matrice[lit[0]][lit[1]]]
            scor *= 2
            
        else:
            score_multiplier = 1
            
            if len(lit_noi) > 1:
                if lit_noi [0][0] == lit_noi [1][0]:
                    scor_horiz = 0
                    for j in range(lit_noi[0][1], 15):
                        if matrice[lit_noi[0][0]][j] == ' ':
                            break
                        if matrice[lit_noi[0][0]][j] != matrice_ant[lit_noi[0][0]][j]:
                            scor_vert = 0
                            cuv_vert = False
                            
                            i = 1
                            while lit_noi[0][0]-i >= 0 and matrice[lit_noi[0][0]-i][j] != ' ':
                                cuv_vert = True
                                scor_vert += puncte[matrice[lit_noi[0][0]-i][j]]
                                i+=1

                            i = 1
                            while lit_noi[0][0]+i < 15 and matrice[lit_noi[0][0]+i][j] != ' ':
                                cuv_vert = True
                                scor_vert += puncte[matrice[lit_noi[0][0]+i][j]]
                                i+=1

                            
                            scor_vert += puncte[matrice[lit_noi[0][0]][j]]
                            scor_horiz += puncte[matrice[lit_noi[0][0]][j]]
                            
                            if tabla[lit_noi[0][0]][j] == 23:
                                score_multiplier *= 3
                                scor_vert *= 3
                            elif tabla[lit_noi[0][0]][j] == 22:
                                score_multiplier *= 2
                                scor_vert *= 2
                            elif tabla[lit_noi[0][0]][j] == 13:
                                scor_horiz += 2 * puncte[matrice[lit_noi[0][0]][j]]
                                scor_vert += 2 * puncte[matrice[lit_noi[0][0]][j]]
                            elif tabla[lit_noi[0][0]][j] == 12:
                                scor_horiz += puncte[matrice[lit_noi[0][0]][j]]
                                scor_vert += puncte[matrice[lit_noi[0][0]][j]]
                            
                            if cuv_vert == True:
                                scor += scor_vert
                                
                        else:
                            scor_horiz += puncte[matrice[lit_noi[0][0]][j]]
                    
                    j = 1
                    while lit_noi[0][1] - j >= 0 and matrice[lit_noi[0][0]][lit_noi[0][1] - j] != ' ':
                        scor_horiz += puncte[matrice[lit_noi[0][0]][lit_noi[0][1] - j]]
                        j+=1
                    
                    scor += scor_horiz * score_multiplier
                    
                elif lit_noi [0][1] == lit_noi [1][1]:
                    scor_vert = 0
                    for i in range(lit_noi[0][0], 15):
                        if matrice[i][lit_noi[0][1]] == ' ':
                            break
                        elif matrice[i, lit_noi[0][1]] != matrice_ant[i][lit_noi[0][1]]:
                            scor_horiz = 0
                            cuv_horiz = False
                            
                            j = 1
                            while lit_noi[0][1]-j >= 0 and matrice[i][lit_noi[0][1]-j] != ' ':
                                cuv_horiz = True
                                scor_horiz += puncte[matrice[i][lit_noi[0][1]-j]]
                                j+=1

                            j = 1
                            while lit_noi[0][1]+j < 15 and matrice[i][lit_noi[0][1]+j] != ' ':
                                cuv_horiz = True
                                scor_horiz += puncte[matrice[i][lit_noi[0][1]+j]]
                                j+=1

                            scor_horiz += puncte[matrice[i][lit_noi[0][1]]]
                            scor_vert += puncte[matrice[i][lit_noi[0][1]]]
                            
                            if tabla[i][lit_noi[0][1]] == 23:
                                score_multiplier *= 3
                                scor_horiz *= 3
                            elif tabla[i][lit_noi[0][1]] == 22:
                                score_multiplier *= 2
                                scor_horiz *= 2
                            elif tabla[i][lit_noi[0][1]] == 13:
                                scor_horiz += 2 * puncte[matrice[i][lit_noi[0][1]]]
                                scor_vert += 2 * puncte[matrice[i][lit_noi[0][1]]]
                            elif tabla[i][lit_noi[0][1]] == 12:
                                scor_horiz += puncte[matrice[i][lit_noi[0][1]]]
                                scor_vert += puncte[matrice[i][lit_noi[0][1]]]
                            
                            if cuv_horiz == True:
                                scor += scor_horiz
                            
                        else:
                            scor_vert += puncte[matrice[i][lit_noi[0][1]]]
                    
                    i = 1
                    while lit_noi[0][0] - i >= 0 and matrice[lit_noi[0][0] - i][lit_noi[0][1]] != ' ':
                        scor_vert += puncte[matrice[lit_noi[0][0] - i][lit_noi[0][1]]]
                        i+=1
                    
                    scor += scor_vert * score_multiplier
                
            else:
                #print("one")
                cuv_vert = False
                cuv_horiz = False

                scor_horiz = puncte[matrice[lit_noi[0][0]][lit_noi[0][1]]]
                scor_vert = puncte[matrice[lit_noi[0][0]][lit_noi[0][1]]]

                i = 1
                while lit_noi[0][0] - i >= 0 and matrice[lit_noi[0][0] - i][lit_noi[0][1]] != ' ':
                    cuv_vert = True
                    scor_vert += puncte[matrice[lit_noi[0][0] - i][lit_noi[0][1]]]
                    i+=1
                i = 1
                while lit_noi[0][0] + i < 15 and matrice[lit_noi[0][0] + i][lit_noi[0][1]] != ' ':
                    cuv_vert = True
                    scor_vert += puncte[matrice[lit_noi[0][0] + i][lit_noi[0][1]]]
                    i+=1

                j = 1
                while lit_noi[0][1] - j >= 0 and matrice[lit_noi[0][0]][lit_noi[0][1] - j] != ' ':
                    cuv_horiz = True
                    scor_horiz += puncte[matrice[lit_noi[0][0]][lit_noi[0][1] - j]]
                    j+=1
                j = 1
                while lit_noi[0][1] + j < 15 and matrice[lit_noi[0][0]][lit_noi[0][1] + j] != ' ':
                    cuv_horiz = True
                    scor_horiz += puncte[matrice[lit_noi[0][0]][lit_noi[0][1] + j]]
                    j+=1

                if tabla[lit_noi[0][0]][lit_noi[0][1]] == 23:
                    scor_vert *= 3
                    scor_horiz *= 3
                elif tabla[lit_noi[0][0]][lit_noi[0][1]] == 22:
                    scor_vert *= 2
                    scor_horiz *= 2
                elif tabla[lit_noi[0][0]][lit_noi[0][1]] == 13:
                    scor_horiz += 2 * puncte[matrice[lit_noi[0][0]][lit_noi[0][1]]]
                    scor_vert += 2 * puncte[matrice[lit_noi[0][0]][lit_noi[0][1]]]
                elif tabla[lit_noi[0][0]][lit_noi[0][1]] == 12:
                    scor_horiz += puncte[matrice[lit_noi[0][0]][lit_noi[0][1]]]
                    scor_vert += puncte[matrice[lit_noi[0][0]][lit_noi[0][1]]]

                if cuv_vert == True:
                    scor += scor_vert
                if cuv_horiz == True:
                    scor += scor_horiz
                        
        matrice_ant = matrice
        index+=1
        index%=20
        afisare_task_3(scor, matrice, lit_noi, file)

# Functii debug

In [32]:
def show_image(title,image):
    image=cv.resize(image,(500,500),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [33]:
def vizualizare_configuratie(result,matrix,lines_horizontal,lines_vertical):
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            if matrix[i][j] == 'x': 
                cv.rectangle(result, (y_min, x_min), (y_max, x_max), color=(255, 0, 0), thickness=5)